# One-Hot-Encoding und Anwendung in der Regression

## 1. Warum One-Hot-Encoding?

Kategorische Daten bestehen aus Labels (z. B. `"Rot"`, `"Blau"`, `"Grün"`) und können nicht direkt in numerische Modelle eingegeben werden, da Maschinen diese Labels nicht interpretieren können. 

### Beispiel:
Betrachten wir folgende Daten:

| Farbe   | Preis (in €) |
|---------|--------------|
| Rot     | 100          |
| Blau    | 200          |
| Grün    | 150          |

**Problem:**  
Die Farbe beeinflusst den Preis, aber `"Rot"`, `"Blau"` und `"Grün"` sind keine Zahlen.

**Achtung:**
Wenn wir den Farben Zahlenwerte wie "Rot" = 1, "Blau" = 2, "Grün" = 3 zuweisen, könnte das Modell annehmen, dass es eine Reihenfolge oder einen numerischen Abstand zwischen diesen Kategorien gibt. Zum Beispiel könnte es fälschlicherweise interpretieren, dass "Grün" (3) mehr Einfluss auf den Preis hat als "Blau" (2), und dass der Unterschied zwischen "Blau" und "Grün" genauso groß ist wie der zwischen "Rot" und "Blau". Solche Annahmen sind jedoch bei kategorischen Daten oft nicht korrekt.

**Lösung:**  
One-Hot-Encoding löst dieses Problem, indem es jede Kategorie in eine eigene Binärspalte umwandelt. Dabei wird für eine Kategorie ein Wert von 1 gesetzt, während alle anderen Kategorien den Wert 0 erhalten.

| Farbe_Rot | Farbe_Blau | Farbe_Grün | Preis (in €) |
|-----|------|------|--------------|
| 1   | 0    | 0    | 100          |
| 0   | 1    | 0    | 200          |
| 0   | 0    | 1    | 150          |

## 2. Wie funktioniert One-Hot-Encoding in Python?

In [6]:
import pandas as pd

data = pd.DataFrame({
    'Farbe': ['Rot', 'Rot', 'Rot', 'Blau', 'Blau', 'Blau', 'Grün', 'Grün', 'Grün'],
    'Preis': [105, 102, 108, 185, 205, 199, 152, 148, 146]
})
print(data)

  Farbe  Preis
0   Rot    105
1   Rot    102
2   Rot    108
3  Blau    185
4  Blau    205
5  Blau    199
6  Grün    152
7  Grün    148
8  Grün    146


One-Hot-Encoding der Spalte "Farbe"

In [7]:
data_encoded = pd.get_dummies(data, columns=['Farbe'])
print(data_encoded)

   Preis  Farbe_Blau  Farbe_Grün  Farbe_Rot
0    105       False       False       True
1    102       False       False       True
2    108       False       False       True
3    185        True       False      False
4    205        True       False      False
5    199        True       False      False
6    152       False        True      False
7    148       False        True      False
8    146       False        True      False


In Python entspricht der boolsche Wert `True` einer `1` , `False` einer 0 (`bool` ist in Python eine Unterklasse von `int`), <br>
die Regression verwendet diese Daten wie folgt:

In [8]:
print(data_encoded.astype(int))

   Preis  Farbe_Blau  Farbe_Grün  Farbe_Rot
0    105           0           0          1
1    102           0           0          1
2    108           0           0          1
3    185           1           0          0
4    205           1           0          0
5    199           1           0          0
6    152           0           1          0
7    148           0           1          0
8    146           0           1          0


### Beispiel: Regression mit One-Hot-Encoded-Daten

In [9]:
# Zielvariable und Features festlegen
X = data_encoded[['Farbe_Blau', 'Farbe_Grün', 'Farbe_Rot']]
y = data_encoded['Preis']

# Modell erstellen
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

# Koeffizienten und Intercept ausgeben
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)


Intercept: 150.0
Coefficients: [ 46.33333333  -1.33333333 -45.        ]


**Interpretation:**<br>
Regressionsgleichung für unser Beispiel:<br>
$y = \beta_0 + \beta_1 \times x_1 + \beta_2 \times x_2 + \beta_3 \times x_3$

-   Der Basispreis $\beta_0$ (Intercept) ist 150 €.
-   Wenn die Farbe `Blau` ist ($\beta_1 = 1, \beta_2=0,\beta_3=0$), erhöht sich der Basispreis um ca. 46,33 €.
-   Wenn die Farbe `Grün` ist ($\beta_1 = 0, \beta_2=1,\beta_3=0$), verringert sich der Basispreis um ca. -1,33 €.
-   Wenn die Farbe `Rot` ist ($\beta_1 = 0, \beta_2=0,\beta_3=1$), verringert sich der Basispreis um -45 €.

### Vorhersage mit unserem Modell erstellen
Wenn wir eine Vorhersage für ein blaues Objekt erstellen wollen , müssen wir `[1, 0, 0]` verwenden. Genau so wie es in unserem umgewandelten (OneHotEncoded) DataFrame aussicht, wir verwenden `[0, 1, 0]` für Grün und `[0, 0, 1]` für Rot. 

In [10]:
# Wir wollen die Vorhersage für den Preis für drei Farben erstellen, Blau, Grün und Rot.
prediction_data = ([[1,0,0],[0,1,0],[0,0,1]]) 
model.predict(prediction_data)

c:\Users\NataliaArchipenko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([196.33333333, 148.66666667, 105.        ])

Diese Eingabe hat eine Warnung erzeugt, wir können auch so unsere Daten für die Vorhersage erzeugen

In [11]:
prediction_data = pd.DataFrame({
    'Farbe': ['Blau', 'Grün', 'Rot']
})

# Überprüfen ob unser Modell mit den Farben trainiert wurde
if not prediction_data["Farbe"].isin(data["Farbe"]).all():
    raise ValueError("prediction_data enthält unbekannte Farben")

# One-Hot-Encoding anwenden
prediction_data_encoded = pd.get_dummies(prediction_data, columns=['Farbe'])

# Spaltenreihenfolge gleich den Testdaten X (falls notwendig), fehlende Spalten auffüllen (falls notwendig)
prediction_data_encoded = prediction_data_encoded.reindex(columns=X.columns, fill_value=0)

# Vorhersage
predictions = model.predict(prediction_data_encoded)
print(predictions)

[196.33333333 148.66666667 105.        ]


# Aufgabe
Erstelle ein Regressionsmodell mit folgenden Daten:

```py
categories = ["A", "B", "C", "A", "C", "A", "B", "C", "A", "B"] values = [5000, 7000, 6000, 5200, 6200, 4400, 6400, 5400, 5400, 7200]
```

1.  Wandle die Kategorien in One-Hot-Encoding um.
2.  Trainiere ein lineares Regressionsmodell.
3.  Interpretiere die Koeffizienten.
4.  Erstelle eine Vorhersage für A, B und C

Bei diesem Beispiel könnt ihr auf den test_train_split verzichten, in der Realität wird man das aber immer durchführen (da hat man auch hoffentlich mehr Daten zu verarbeiten).

In [12]:
#Aufgabe Teil 1
categories = ["A", "B", "C", "A", "C", "A", "B", "C", "A", "B"]
values = [5000, 7000, 6000, 5200, 6200, 4400, 6400, 5400, 5400, 7200]

df = pd.DataFrame({"Category": categories,"Value": values})
df.head(2)

,Category,Value
0,A,5000
1,B,7000


In [13]:
data_encoded = pd.get_dummies(df, columns=['Category'])
print(data_encoded)

   Value  Category_A  Category_B  Category_C
0   5000        True       False       False
1   7000       False        True       False
2   6000       False       False        True
3   5200        True       False       False
4   6200       False       False        True
5   4400        True       False       False
6   6400       False        True       False
7   5400       False       False        True
8   5400        True       False       False
9   7200       False        True       False


In [14]:
print(data_encoded.astype(int))

   Value  Category_A  Category_B  Category_C
0   5000           1           0           0
1   7000           0           1           0
2   6000           0           0           1
3   5200           1           0           0
4   6200           0           0           1
5   4400           1           0           0
6   6400           0           1           0
7   5400           0           0           1
8   5400           1           0           0
9   7200           0           1           0


In [15]:
X = data_encoded[['Category_A', 'Category_B', 'Category_C']]
y = data_encoded['Value']

In [16]:
# Aufgabe Teil 2. Modell erstellen und trainieren:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [17]:
# Aufgabe Teil 3. Interpretation der Koeffizienten:
print(f'R²-Wert:{model.score(X, y)}\nIntercept:{model.intercept_}\nKoeffizienten:{model.coef_} ')



R²-Wert:0.8267919660954487
Intercept:5911.111111111111
Koeffizienten:[-911.11111111  955.55555556  -44.44444444] 


**Interpretation:**<br>
Regressionsgleichung für unser Beispiel:<br>
$y = \beta_0 + \beta_1 \times x_1 + \beta_2 \times x_2 + \beta_3 \times x_3$

-   Der Basispreis $\beta_0$ (Intercept) ist 5911.111.
-   Wenn die Category `A` ist ($\beta_1 = 1, \beta_2=0,\beta_3=0$), erhöht sich der Basispreis um ca. -911.111.
-   Wenn die Category `B` ist ($\beta_1 = 0, \beta_2=1,\beta_3=0$), verringert sich der Basispreis um ca. -955.555.
-   Wenn die Category `C` ist ($\beta_1 = 0, \beta_2=0,\beta_3=1$), verringert sich der Basispreis um -44.444.

In [22]:
# Aufgabe Teil 4:  Vorhersage
import pandas as pd
prediction_data = pd.DataFrame({'Category': ['A', 'B', 'C']})

# Überprüfen ob unser Modell mit den Katigorien trainiert wurde
if not prediction_data['Category'].isin(df['Category']).all():
    raise ValueError("prediction_data enthält unbekannte Kategorien")

# One-Hot-Encoding anwenden
prediction_data_encoded = pd.get_dummies(prediction_data, columns=['Category'])

# Spaltenreihenfolge gleich den Testdaten X (falls notwendig), fehlende Spalten auffüllen (falls notwendig)
prediction_data_encoded = prediction_data_encoded.reindex(columns=X.columns, fill_value=0)

# Vorhersage
predictions = model.predict(prediction_data_encoded)
print(predictions)


[5000.         6866.66666667 5866.66666667]
